# Daily Stopwatch Data Science: Two Sigma Financial Modeling Challenge

Note: this is a part of a Kaggle competition: Two Sigma Financial Modeling Challenge. For more information, see https://www.kaggle.com/c/two-sigma-financial-modeling

## Stage 1: Ask a question

My objective is to the predict the value of unknown variable $y$, which is a time series.

I measure the performance by R-squared. It seems that the actual competition has online version of it for reinforcement learning. I will get back to it later.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

In [64]:
#import sys
#reload(sys)
#sys.setdefaultencoding("unicode")

#Set up the environment
import numpy as np                         #Numpy
import pandas as pd                        #Pandas
import matplotlib.pyplot as plt            #Plot

In [65]:

# Set up data directory
DataDir = "C:/Users/Admin/Documents/data/"

# Here's an example of loading the CSV using Pandas's built-in HDF5 support:
with pd.HDFStore(DataDir+ "train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

In [66]:
%matplotlib inline                         
#Make plot appeared inline

## Stage 3: Explore the data

Explore, Visualize, Clean, Transform, Feature engineering

In [67]:
#Basis stats
len(df), len(df.columns)   #number of rows and columns

(1710756, 111)

In [68]:
#See all column names and types
#print(df.dtypes.to_string())

In [69]:
#See first ten rows
#df.head(10)

In [70]:
#See last ten rows
#df.tail(10)

Here one can see something peculiar about this data. There are ids and timestamps. Let's check the number of unique elements

In [71]:
len(df["timestamp"].unique()), len(df["id"].unique())

(1813, 1424)

Let's create a summary table for those columns including type, min, mean, max, sigma, percent of zeros, and percent of missing values

In [72]:
df_column_summary = pd.DataFrame(df.dtypes,columns=['type'])
df_column_summary.reset_index(inplace=True)
df_column_summary['min'] = list(df.min())
df_column_summary['mean'] = list(df.mean())
df_column_summary['max'] = list(df.max())
df_column_summary['sigma'] = list(df.std())

l = ['NA'] * len(df.columns)
for i in range(0,len(df.columns)):
    l[i] = 1.0-np.count_nonzero(df.iloc[:,i])*1.0/len(df)
df_column_summary['zero'] = l

df_column_summary['missing'] = list(1.0-df.count()*1.0/len(df))

In [73]:
print(df_column_summary.to_string())

              index     type           min          mean           max         sigma      zero   missing
0                id    int16  0.000000e+00  1.093369e+03  2.158000e+03  6.306703e+02  0.000962  0.000000
1         timestamp    int16  0.000000e+00  9.455825e+02  1.812000e+03  5.195463e+02  0.000438  0.000000
2         derived_0  float32 -2.017497e+04 -4.536046e+00  3.252527e+03  2.497382e+02  0.000000  0.042647
3         derived_1  float32 -7.375435e-02  7.729436e+11  1.068448e+16  7.620606e+13  0.000000  0.047364
4         derived_2  float32 -9.848880e+03 -3.320328e-01  3.823001e+03  6.519810e+01  0.000000  0.233026
5         derived_3  float32 -3.434176e+04 -5.046012e-01  1.239737e+03  1.020749e+02  0.000000  0.087371
6         derived_4  float32 -8.551914e+03  1.801661e+01  6.785965e+04  9.258360e+02  0.000000  0.237590
7     fundamental_0  float32 -2.344957e+00 -2.040938e-02  1.378195e+00  2.494859e-01  0.000000  0.013998
8     fundamental_1  float32 -1.043737e+13 -5.703754e+0

Here are some lessons from data summary:

1. It makes sense to treat this id as an object rather than int to avoid the sense of ordering. 
2. In terms of sigma (together with other measures), one can spot a strange large number in many features. Worth taking a look.
3. In terms of max and min, one can spot discreteness in many features. It may be non-numerical. Worth taking a look.
4. In terms of zero values, one should aware that zero may mean missing values. It is also associated with a discrete feature.
5. In terms of missing values, there is nothing to worry yet. No significant poor features.

Let's further investigate each feature using visualization tool.

In [74]:
#Play around with interactive visualization tool. However, due to large amount of data. Let's use simple one instead.
#from bokeh.plotting import figure, output_notebook, show
#output_notebook()

#i = 0
#p = figure(title=df.columns[i], width=500, height=500)
#p.circle(df[df.columns[i]], df['y'], size=7, color="firebrick", alpha=0.5)
#show(p)

According to the notes above, we should look into feature 3, 8, 23, 67 (large sigma)

In [75]:
#2-d array plots
#l = [3, 8, 23, 67]
#f, axarr = plt.subplots(1, 4, figsize=(4*4,3*1))
#num = 0
#for i in l:
#    axarr[num].scatter(df[df.columns[i]],df['y'])
#    axarr[num].set_title(str(i))
#    num = num + 1
                             

According to the notes above, we should look into feature 70, 72, 75, 77, 78, 79, 80, 82, 83, 84, 85, 89, 94, 97, 99, 102, 103, 104, 107, 108 (discrete max/min)

In [76]:
#2-d array plots
#l = [70, 72, 75, 77, 78, 79, 80, 82, 83, 84, 85, 89, 94, 97, 99, 102, 103, 104, 107, 108]
#f, axarr = plt.subplots(5, 4, figsize=(4*4,3*5))
#num = 0
#for i in l:
#    axarr[num/4,num%4].scatter(df[df.columns[i]],df['y'])
#    axarr[num/4,num%4].set_title(str(i))
#    num = num + 1
                             

According to the notes above, we should look into feature (70), (77), (78), (80), 81, (82), (83), 85, 87, (89), 94, 95, (97), (99), (102), (103), (104), (107) (# of zeros > 10 percent)

In [77]:
#2-d array plots
#l =  [81, 85, 87, 94, 95]
#f, axarr = plt.subplots(2, 4, figsize=(4*4,3*2))
#num = 0
#for i in l:
#    axarr[num/4,num%4].scatter(df[df.columns[i]],df['y'])
#    axarr[num/4,num%4].set_title(str(i))
#    num = num + 1
                             

These features do not give a strong signal but it looks fine. There are only two discrete features 89 and 99. Sinee they are set to be decimal points, let's believe that there are ordering features. So, there is nothing to worry for now.

Let's normalize data so that each feature has 0-mean and 1-std (except id, timestamp, and input). 

In [78]:
#One may use sklearn to do this, but the trouble is we need to deal with NA values, which is annoying. So just do it from scratch.
#from sklearn.preprocessing import normalize
#X = normalize(np.array(df[df.columns[range(2,110)]].dropna(axis=0)))

In [79]:
for i in range(2,110):
    mean = df_column_summary.loc[i,'mean']
    sigma = df_column_summary.loc[i,'sigma']
    df[df.columns[i]] = (df[df.columns[i]]-mean)/sigma

Next let's look at possible dimensionality reduction for features (except id, timestamp, and output).

In [80]:
#Let's try PCA
from sklearn.decomposition import PCA

In [81]:
#try group them according to feature types: derived
X = np.array(df[df.columns[range(2,7)]].dropna(axis=0))
pca_derived = PCA(n_components=5)
pca_derived.fit(X)
print(pca_derived.explained_variance_ratio_) 

[ 0.32672635  0.27849502  0.27681062  0.1114164   0.00655162]


In [82]:
#try group them according to feature types: fundamental
X = np.array(df[df.columns[range(7,70)]].dropna(axis=0))
pca_fundamental = PCA(n_components=20)
pca_fundamental.fit(X)
print(pca_fundamental.explained_variance_ratio_) 

[ 0.25558325  0.13097034  0.10843365  0.06818235  0.06199728  0.05705869
  0.04216031  0.03775116  0.03570373  0.03272416  0.02860421  0.02629839
  0.02333962  0.02214361  0.01652564  0.01068863  0.00874587  0.00707701
  0.00565645  0.00433277]


In [83]:
#try group them according to feature types: technical
X = np.array(df[df.columns[range(70,110)]].dropna(axis=0))
pca_technical = PCA(n_components=40)
pca_technical.fit(X)
print(pca_technical.explained_variance_ratio_) 

[ 0.144173    0.08449022  0.05374959  0.04201616  0.03940461  0.03401835
  0.02991891  0.02958015  0.02731697  0.02642108  0.02606273  0.02566239
  0.02549551  0.0254263   0.02527166  0.02518373  0.02511065  0.02498515
  0.02477049  0.02456923  0.0228427   0.02177022  0.02136387  0.01724815
  0.01452915  0.01372289  0.01339999  0.01210162  0.0116663   0.0110958
  0.01007953  0.00972421  0.00956461  0.00916052  0.00891192  0.00738879
  0.00706167  0.00642801  0.00502863  0.00328453]


Let's use a rule of thumbs, say, we want to capture at least 95% variance ratio. For 'derived', we need 4 out of 5 components. For 'fundamental', we need 16 out of 63 components. For 'technical', we need 32 out of 40 components.  

The problem with PCA is that we need to deal with NA values. If we want to continue analysis with PCA, we may 

1. fill NA with zero (mean) and continue with full number of rows. 
2. delete rows with NA and proceed.

Since there are many NAs in data, let's try option 1 and see how it goes.

In [84]:
#CLEAN
#replace NaN values with zero (mean).
df = df.fillna(value=0)

In [85]:
d5.columns

Index([u'y'], dtype='object')

In [86]:
#TRANFORM/ NEW VARIABLES
'''
#Apply PCA to different group of features and reattached everything
d1 = df[['id','timestamp']]
#derived
d2 = pd.DataFrame(pca_derived.transform(df[df.columns[range(2,7)]]))
d2 = d2.loc[:,0:3]
d2.columns = ['derived_pca_1','derived_pca_2','derived_pca_3','derived_pca_4']

d3 = pd.DataFrame(pca_fundamental.transform(df[df.columns[range(7,70)]]))
d3 = d3.loc[:,0:15]
d3.columns = ['fundamental_pca_' + str(i) for i in range(0,16)]

d4 = pd.DataFrame(pca_technical.transform(df[df.columns[range(70,110)]]))
d4 = d4.loc[:,0:31]
d4.columns = ['technical_pca_' + str(i) for i in range(0,32)]

d5 = df[['y']]

df = pd.concat([d1,d2,d3,d4,d5])
'''
#It seems that using this new data, the model suffers from too much information. We need new ways to address this issue.

# Convert ID to be object, not a number
df['id'] = df['id'].astype(object)

## Stage 4: Model the data

I have prepared data for validation as follow:

In [87]:
# Now test/train split by random selection. Ideally, we should do cross-validation and parameter average, but save it for later.
r = np.random.uniform(0,1,len(df)) # Random UNIForm numbers, one per row
train = df[ r < 0.7]
test = df[0.7 <= r]

In [88]:
len(train), len(test)

(1197427, 513329)

First, let's try something simple that accommodate non-number types: trees.

In [89]:
#Random forest
#from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

X_train = train.drop('y', axis=1)
y_train = train['y']
X_test = test.drop('y', axis=1)
y_test = test['y']

In [90]:
#clf = RandomForestRegressor()
#clf = clf.fit(X_train,y_train)

#Took too much time to respond

Here is the plan. Let's do something called grand PCA. That is, we make one row to represent each timestamp. Then we perform PCA from there on both inputs and outputs. Then see how it goes.

*Note to myself*: The code does not work (memory and speed issue). I spent roughly 2 Pomodoros. Let's restart tomorrow.